# Anselm — QLoRA fine-tuning with Axolotl (Colab-friendly)

This notebook performs QLoRA fine-tuning of a Mistral-7B-Instruct base model using Axolotl. It:
- Installs dependencies (PyTorch + Axolotl + bitsandbytes + peft)
- Mounts Google Drive (optional) and prepares `data/processed/`
- Writes `configs/anselm_qlora.yaml` with QLoRA + LoRA settings
- Runs training via `accelerate launch -m axolotl.cli.train configs/anselm_qlora.yaml`
- Shows TensorBoard for monitoring and includes a GPU monitor cell
- Demonstrates loading the LoRA adapter for inference

Run cells top-to-bottom. Select a GPU runtime in Colab (Runtime → Change runtime type → GPU).

In [ ]:
# 1) Environment check: GPU, Python, and basic info
import os, sys, subprocess
print('Python', sys.version)
try:
    gpu_info = subprocess.check_output(['nvidia-smi','--query-gpu=name,memory.total --format=csv,noheader,nounits'], text=True)
    print('GPU info:', gpu_info)
except Exception as e:
    print('nvidia-smi not available or no GPU: ', e)

print('Current working directory:', os.getcwd())

In [ ]:
# 2) Install dependencies (Colab-friendly). Run once.
# Adjust torch wheel index if you need a different CUDA version.
print('Installing dependencies (may take several minutes)')
!pip install -q --upgrade pip
!pip install -q 'torch' torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q accelerate axolotl transformers bitsandbytes peft tensorboard sentence-transformers python-dotenv

In [ ]:

print('Install completed. Verifying key packages...')
import importlib
for pkg in ('torch','accelerate','axolotl','transformers','bitsandbytes','peft','tensorboard'):
    try:
        m = importlib.import_module(pkg)
        print(pkg, 'version', getattr(m, '__version__', 'unknown'))
    except Exception as e:
        print(pkg, 'import error:', e)

In [ ]:
# 3) Mount Google Drive if running on Colab (optional). If not on Colab, skip and ensure your data exists under data/processed/
try:
    import google.colab
    from google.colab import drive
    print('Running on Colab — mounting Drive to /content/drive')
    drive.mount('/content/drive')
except Exception:
    print('Not running on Colab or google.colab not available. If running locally, ensure your dataset is at data/processed/ or upload it.')

In [ ]:
# 4) Prepare directories and copy dataset from Drive if present
from pathlib import Path
import shutil
content_data = Path('/content/data/processed')
content_data.mkdir(parents=True, exist_ok=True)
print('Ensured', content_data)
drive_path = Path('/content/drive/MyDrive/almost-anselm/data/processed')
if drive_path.exists():
    print('Found dataset in Drive — copying to', content_data)
    for p in drive_path.glob('*'):
        dst = content_data / p.name
        if dst.exists():
            print('Skipping existing', dst)
            continue
        if p.is_dir():
            shutil.copytree(p, dst)
        else:
            shutil.copy2(p, dst)
    print('Copy complete')
else:
    print('Drive dataset not found at', drive_path, 'Make sure your train/val/test files are in', content_data)
print('Current data/processed contents:')
print(list(content_data.glob('*')))

In [ ]:
# 5) Write the Axolotl config file for QLoRA: configs/anselm_qlora.yaml
from pathlib import Path
cfg_dir = Path('configs')
cfg_dir.mkdir(parents=True, exist_ok=True)
cfg_path = cfg_dir / 'anselm_qlora.yaml'
cfg_text = '''
base_model: mistral-7b-instruct-v0.3
model_type: mistral
load_in_4bit: true
trust_remote_code: true
torch_dtype: float16
dataset:
  train: data/processed/sft_train.json
  val: data/processed/sft_val.json
  test: data/processed/sft_test.json
dataset_format: chatml
tokenizer:
  add_eos_token: true
  add_bos_token: true
sequence:
  max_length: 2048
  sample_packing: true
training:
  output_dir: models/almost-anselm-lora/
  num_epochs: 1
  per_device_train_batch_size: 1
  gradient_accumulation_steps: 16
  learning_rate: 1e-4
  lr_scheduler: cosine
  warmup_steps: 100
  logging_steps: 50
  eval_steps: 1000
  save_steps: 1000
  save_total_limit: 3
  bf16: false
  fp16: true
  gradient_checkpointing: true
  optim: paged_adamw_32bit
lora:
  enabled: true
  r: 16
  lora_alpha: 32
  lora_dropout: 0.05
  target_modules:
    - q_proj
    - k_proj
    - v_proj
    - o_proj
'''
cfg_path.write_text(cfg_text)
print('Wrote', cfg_path)

In [ ]:
# 6) Print the config to verify
print(open('configs/anselm_qlora.yaml','r').read())

## 7) Run training (Accelerate + Axolotl)

The command below runs Axolotl training using `accelerate`. It writes stdout/stderr to `logs/axolotl_train.log` so you can inspect progress after starting. On Colab, running interactively will stream logs into the notebook output. If you prefer to run in background, use the `nohup` example provided.

In [ ]:
# Create logs directory
from pathlib import Path
Path('logs').mkdir(exist_ok=True)
print('When ready, run (uncomment) the accelerate command in this cell to start training.')
# Example interactive run:
# !accelerate launch -m axolotl.cli.train configs/anselm_qlora.yaml 2>&1 | tee logs/axolotl_train.log
# Example background run:
# !nohup accelerate launch -m axolotl.cli.train configs/anselm_qlora.yaml > logs/axolotl_train.log 2>&1 &

In [ ]:
# 8) TensorBoard — visualize training metrics (run after logs exist)
try:
    get_ipython().run_line_magic('load_ext','tensorboard')
except Exception:
    pass
logdirs = ['runs','logs','models/almost-anselm-lora/runs']
from pathlib import Path
existing = [p for p in logdirs if Path(p).exists()]
if not existing:
    print('No TensorBoard logs found yet in', logdirs)
else:
    print('Starting TensorBoard for', existing[0])
    get_ipython().run_line_magic('tensorboard', f'--logdir {existing[0]} --host 0.0.0.0 --port 6006')

In [ ]:
# 9) Tailing logs & simple GPU monitor (run interactively; interrupt to stop)
import time, subprocess
from pathlib import Path
log_path = Path('logs/axolotl_train.log')
print('Tailing', log_path)
try:
    while True:
        if log_path.exists():
            tail = subprocess.run(['tail','-n','20', str(log_path)], capture_output=True, text=True)
            print('--- last 20 lines of log ---')
            print(tail.stdout)
        else:
            print('Log not found yet at', log_path)
        try:
            g = subprocess.check_output(['nvidia-smi','--query-gpu=index,name,memory.total,memory.used,utilization.gpu --format=csv,noheader,nounits'], text=True)
            print('GPU:')
            print(g)
        except Exception:
            print('nvidia-smi not available')
        time.sleep(10)
except KeyboardInterrupt:
    print('Stopped tailing/log monitor')

In [ ]:
# 10) After training: inspect or copy adapter output into models/almost-anselm-lora/
from pathlib import Path
outdir = Path('models/almost-anselm-lora')
outdir.mkdir(parents=True, exist_ok=True)
print('Contents of models/ (top-level):')
print(list(Path('models').glob('*')))

## 11) Inference: Load base model (4-bit) and the trained LoRA adapter

This cell uses `transformers`, `bitsandbytes`, and `peft` to load the base model with 4-bit quantization and then load the LoRA adapter from `models/almost-anselm-lora/`. It runs a few sample prompts.

In [ ]:
# 11a) Inference loader and generator
import os
from pathlib import Path
adapter_path = Path('models/almost-anselm-lora')
if not adapter_path.exists() or not any(adapter_path.iterdir()):
    print('Adapter dir appears empty; ensure you copied adapter checkpoints to', adapter_path)
else:
    print('Adapter path:', adapter_path)
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM
    from peft import PeftModel
    import torch
except Exception as e:
    print('Missing packages for inference:', e)
    raise
MODEL_NAME = 'mistral-7b-instruct-v0.3'
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
print('Loading 4-bit model (this may take a while)...')
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, load_in_4bit=True, device_map='auto', trust_remote_code=True)
print('Base model loaded')
print('Attempting to load LoRA adapter from', adapter_path)
model = PeftModel.from_pretrained(model, adapter_path, device_map='auto')
print('Adapter loaded successfully')
def generate(prompt, max_new_tokens=128, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=temperature)
    return tokenizer.decode(out[0], skip_special_tokens=True)
prompts = [
    'Write a short reflective reply to: I had a rough day and could use some advice.',
    'Explain briefly why patience matters in conversation.'
]
for p in prompts:
    print('
PROMPT:', p)
    print('REPLY:', generate(p))

## 12) CLI command (run outside notebook)

Use this command from the repository root (ensure `accelerate` config is set up):
```
accelerate launch -m axolotl.cli.train configs/anselm_qlora.yaml
```
You can redirect logs to a file:
```
accelerate launch -m axolotl.cli.train configs/anselm_qlora.yaml 2>&1 | tee logs/axolotl_train.log
```

## Troubleshooting & Notes

- CUDA / Torch mismatch: If `torch` fails to import or GPU isn't available, install the matching torch wheel for your CUDA version. In Colab, CUDA 11.8 wheels usually work.
- OOM: If you run out of memory, reduce `per_device_train_batch_size` or increase gradient accumulation. You can also use `load_in_4bit: true` (already enabled) and `gradient_checkpointing: true`.
- `trust_remote_code: true` is convenient but review code for untrusted checkpoints.
- If LoRA adapter doesn't load, inspect the saved checkpoint folder and ensure the adapter files (adapter_config.json, adapter_model.bin/weights) exist. The `peft` loader expects a compatible layout.
- To reproduce training parameters outside Colab, run the `accelerate launch` command above after configuring `accelerate config`.